In [91]:
# imports
from sympy import diff, symbols, exp, Eq, solve, lambdify
from sympy.abc import t, s
import sympy

import numpy as np
import matplotlib.pyplot as plt

from scipy.optimize import fsolve
from dataclasses import dataclass, field

import control

# State Space Model:

$$\frac{dC_A}{dt}=\frac{F}{V}\left(C_{A0}-C_A\right)-k_1(T)C_A\left(C_A+C_{B0}-C_{A0}\right)$$

$$\frac{dC_S}{dt}=\frac{F}{V}\left(C_{S0}-C_S\right)-k_2(T)C_S$$

$$\frac{dT}{dt}=\frac{F}{V}\left(T_0-T\right)+\frac{-\Delta H_1k_1(T)C_A\left(C_A+C_{B0}-C_{A0}\right)-\Delta H_2k_2(T)C_S}{\rho C_p}-\frac{UA\left(T-T_c\right)}{V\rho C_p}$$

## Define symbolic state space

In [2]:
# define symbolic state space equations
F, V, C_A0, C_A, k_1, T, C_A, C_B0, k_10, R, E_1, E_2, C_S0, C_S, E_2, k_20, H_1, H_2, T_0, T_c, UA, rho, C_p = symbols('F, V, C_A0, C_A, k_1, T, C_A, C_B0, k_10, R, E_1, E_2, C_S0, C_S, E_2, k_20, H_1, H_2, T_0, T_c, UA, rho, C_p')

C_As, C_Bs, C_Cs, C_Ss = symbols('C_As, C_Bs, C_Cs, C_Ss')

k_1 = k_10 * exp(-E_1 / T / R)
k_2 = k_20 * exp(-E_2 / T / R)

f_C_A = F / V * (C_A0 - C_A) - k_1 * C_A * (C_A + C_B0 - C_A0)

f_C_S = F / V * (C_S0 - C_S) - k_2 * C_S

f_T = F / V * (T_0 - T) + (-H_1 * k_1 * C_A * (C_A + C_B0 - C_A0) - H_2 * k_2 * C_S) / (rho * C_p) - (UA * (T - T_c)) / (V * rho * C_p)

### Verify Equations

In [3]:
f_C_A

-C_A*k_10*(C_A - C_A0 + C_B0)*exp(-E_1/(R*T)) + F*(-C_A + C_A0)/V

In [4]:
f_C_S

-C_S*k_20*exp(-E_2/(R*T)) + F*(-C_S + C_S0)/V

In [5]:
f_T

F*(-T + T_0)/V - UA*(T - T_c)/(C_p*V*rho) + (-C_A*H_1*k_10*(C_A - C_A0 + C_B0)*exp(-E_1/(R*T)) - C_S*H_2*k_20*exp(-E_2/(R*T)))/(C_p*rho)

# Define system paramters

## Classes that define the system

In [6]:
# system classes
@dataclass
class SystemParameters():
    k_10: float
    E_1: float 
    k_20: float
    E_2: float
    H_1: float
    H_2: float
    rho: float
    C_p: float
    A: float
    T_c: float
    V: float

@dataclass
class InletConditions():
    F_A0: float
    F_S0: float
    F_B0: float
    v_0: float = field(init=False)
    C_A0: float = field(init=False)
    C_B0: float = field(init=False)
    C_S0: float = field(init=False)
    F: float = field(init=False)

    def __post_init__(self):
        self.F = self.F_A0 + self.F_B0 + self.F_S0

@dataclass
class SteadyState():
    U_s: float
    T_0s: float
    C_As: float = field(init=False)
    C_Ss: float = field(init=False)
    T_s: float = field(init=False)

## Initilize classes with given parameters

In [7]:
# initialize system classes
System_Params = SystemParameters(**{
    "k_10": 4e14,
    "E_1": 1.28e5,  
    "k_20": 1e84,
    "E_2": 8e5,
    "H_1": -45400,
    "H_2": -3.2e5,
    "rho": 12.4,
    "C_p": 254,
    "A": 5.3,
    "T_c": 373,
    "V": 4000,
})

Inlet_Conds = InletConditions(**{
    "F_A0": 1075,
    "F_S0": 750,
    "F_B0": 1275,
})

Inlet_Conds.v_0 = Inlet_Conds.F / System_Params.rho
Inlet_Conds.C_A0 = Inlet_Conds.F_A0 / Inlet_Conds.v_0
Inlet_Conds.C_B0 = Inlet_Conds.F_B0 / Inlet_Conds.v_0
Inlet_Conds.C_S0 = Inlet_Conds.F_S0 / Inlet_Conds.v_0

Steady_States = SteadyState(**{
    "U_s": 14.656e3,
    "T_0s": 410,
})

In [10]:
Inlet_Conds

InletConditions(F_A0=1075, F_S0=750, F_B0=1275, v_0=250.0, C_A0=4.3, C_B0=5.1, C_S0=3.0, F=3100)

## Substitue parameters into state space

In [80]:
# substitute system paramters into state space
f_C_A_subbed = f_C_A.subs({
    C_A0: Inlet_Conds.C_A0,
    k_10: System_Params.k_10,
    R: 8.314,
    F: Inlet_Conds.v_0,
    V: System_Params.V,
    E_1: System_Params.E_1,
    C_B0: Inlet_Conds.C_B0,
})

f_C_S_subbed = f_C_S.subs({
    C_A0: Inlet_Conds.C_A0,
    k_10: System_Params.k_10,
    R: 8.314,
    F: Inlet_Conds.v_0,
    V: System_Params.V,
    E_2: System_Params.E_2,
    C_S0: Inlet_Conds.C_S0,
    k_20: System_Params.k_20,
    C_B0: Inlet_Conds.C_B0,
})

f_T_subbed = f_T.subs({
    C_A0: Inlet_Conds.C_A0,
    k_10: System_Params.k_10,
    R: 8.314,
    F: Inlet_Conds.v_0,
    V: System_Params.V,
    E_1: System_Params.E_1,
    E_2: System_Params.E_2,
    C_S0: Inlet_Conds.C_S0,
    k_20: System_Params.k_20,
    UA: Steady_States.U_s * System_Params.A,
    T_c: System_Params.T_c,
    H_1: System_Params.H_1,
    H_2: System_Params.H_2,
    C_p: System_Params.C_p, 
    rho: System_Params.rho,
    C_B0: Inlet_Conds.C_B0,
})

In [81]:
f_C_A_subbed

-400000000000000.0*C_A*(C_A + 0.8)*exp(-15395.7180659129/T) - 0.0625*C_A + 0.26875

In [82]:
f_C_S_subbed

-0.0625*C_S - 1.0e+84*C_S*exp(-96223.2379119557/T) + 0.1875

In [83]:
f_T_subbed

5.76581153162306e+15*C_A*(C_A + 0.8)*exp(-15395.7180659129/T) + 1.01600203200406e+86*C_S*exp(-96223.2379119557/T) - 0.0686656083312167*T + 0.0625*T_0 + 2.29977190754382

# Evaluate steady state

In [84]:
def solve_C_A(_T):
    _f = lambdify(C_A, f_C_A_subbed.subs({T: _T}), 'numpy')
    _C_A = fsolve(_f, [4])

    return _C_A[0]

def solve_C_S(_T):
    _f = lambdify(C_S, f_C_S_subbed.subs({T: _T}), 'numpy')
    _C_S = fsolve(_f, [4])

    return _C_S[0]

def solve_heat(_T):
    _C_A = solve_C_A(_T)
    _C_S = solve_C_S(_T)
    __f = f_T_subbed.subs({C_A: _C_A, C_S: _C_S, T_0: Steady_States.T_0s})
    _f = lambdify(T, __f, 'numpy')
    return _f(_T)

def root_solve(f, x0):
    x_n = x0
    for _ in range(100):
        dx = 1e-12
        diff = (f(x_n + dx) - f(x_n)) / dx
        x_n = x_n - f(x_n) / diff

        if np.abs(diff) < 1e-10:
            break

    return x_n

Steady_States.T_s = root_solve(solve_heat, 460)

In [85]:
# save steady states
Steady_States.C_As = solve_C_A(Steady_States.T_s)
Steady_States.C_Ss = solve_C_S(Steady_States.T_s)
Steady_States

SteadyState(U_s=14656.0, T_0s=410, C_As=0.2116701379629191, C_Ss=2.9999920948240835, T_s=460.3182315699554)

# Steady state conversion

In [86]:
# A conversion
1 - Steady_States.C_As / Inlet_Conds.C_A0

0.9507743865202514

In [87]:
# S conversion
1 - Steady_States.C_Ss / Inlet_Conds.C_S0

2.6350586388446118e-06

### Linearization

In [88]:
# linear matrices
A = sympy.matrices.Matrix([
    [
        f_C_A_subbed.diff(C_A).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s}), 
        f_C_A_subbed.diff(C_S).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s}), 
        f_C_A_subbed.diff(T).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s})
    ],
    [
        f_C_S_subbed.diff(C_A).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s}), 
        f_C_S_subbed.diff(C_S).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s}), 
        f_C_S_subbed.diff(T).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s})
    ],
    [
        f_T_subbed.diff(C_A).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s}), 
        f_T_subbed.diff(C_S).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s}), 
        f_T_subbed.diff(T).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s})
    ],
])

b = sympy.matrices.Matrix([
    [f_C_A_subbed.diff(T_0).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s})],
    [f_C_S_subbed.diff(T_0).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s})],
    [f_T_subbed.diff(T_0).subs({C_A: Steady_States.C_As, C_S: Steady_States.C_Ss, T: Steady_States.T_s})],
])

c = sympy.matrices.Matrix([
    [0, 0, 1],
])

d = 0

state_vector = sympy.matrices.Matrix([
    [C_A],
    [C_S],
    [T],
])

In [101]:
A

Matrix([
[-1.52223727109086,                   0,  -0.0185656237024397],
[                0, -0.0625001646915989, -2.24365031604688e-7],
[ 21.0414249769892, 1.67326999164773e-5,    0.198971905289839]])

In [90]:
b

Matrix([
[     0],
[     0],
[0.0625]])

In [102]:
c

Matrix([[0, 0, 1]])

$$G(s)=c\left(sI-A\right)^{-1}+d$$

In [99]:
(c * (s * sympy.matrices.eye(3) - A)**-1 * b + sympy.matrices.Matrix([d]))[0]

0.0625*(1.0*s**2 + 1.58473743578246*s + 0.0951400801428692)/(1.0*s**3 + 1.38576553049263*s**2 + 0.170469031450751*s + 0.00548530996943723)